# API Method1

In [ ]:
# import requests
# import pandas as pd
# import os

# from dotenv import load_dotenv
# load_dotenv()



# def fetch_data(base_url, dataset, api_key, num_records=99, offset=0):
#     all_records = []
#     max_offset = 9900  # Maximum number of requests

#     while True:
#         # maximum limit check
#         if offset > max_offset:
#             break

#         # Create API request URL
#         filters = f'{dataset}/records?limit={num_records}&offset={offset}'
#         url = f'{base_url}{filters}&api_key={api_key_mop}'

#         # Start request
#         try:
#             result = requests.get(url, timeout=10)
#             result.raise_for_status()
#             records = result.json().get('results')
#         except requests.exceptions.RequestException as e:
#             raise Exception(f"API request failed: {e}")
#         if records is None:
#             break
#         all_records.extend(records)
#         if len(records) < num_records:
#             break

#         # next cycle offset
#         offset += num_records

#     # DataFrame all data
#     df = pd.DataFrame(all_records)
#     return df
    
# API_KEY = os.environ.get("API_KEY_MOP")
# BASE_URL = 'https://melbournetestbed.opendatasoft.com/api/explore/v2.1/catalog/datasets/'


In [ ]:
# # Microclimate Sensor Readings
# CLIMATE_DATASET = 'microclimate-sensor-readings'
# microclimate_readings = fetch_data(BASE_URL, CLIMATE_DATASET, API_KEY)
# microclimate_readings

In [ ]:
# #Microclimate Sensor Locations
# SENSOR_LOCATIONS_DATASET = 'microclimate-sensor-locations'
# climate_sensor_locations = fetch_data(BASE_URL, SENSOR_LOCATIONS_DATASET, API_KEY)
# climate_sensor_locations


In [ ]:
# #Footpath steepness
# FOOTPATH_STEEPNESS_DATASET = 'footpath-steepness'
# footpath_steepness = fetch_data(BASE_URL, FOOTPATH_STEEPNESS_DATASET, API_KEY)
# footpath_steepness

In [ ]:
# #Pedestrian Counting System - Past Hour (counts per minute)
# PEDESTRIAN_DATASET = 'pedestrian-counting-system-past-hour-counts-per-minute'
# pedestraian_count = fetch_data(BASE_URL, PEDESTRIAN_DATASET, API_KEY)
# pedestraian_count

In [ ]:
# #Pedestrian Counting System - Sensor Locations
# SENSOR_LOCATIONS_DATASET_2 = 'pedestrian-counting-system-sensor-locations'
# pedestrian_sensor_locations = fetch_data(BASE_URL, SENSOR_LOCATIONS_DATASET_2, API_KEY)
# pedestrian_sensor_locations

In [ ]:
# #datasets_url = "?dataset=global-summary-of-the-day"

# #NOAA Dataset

# import requests
# import pandas as pd

# # Define the base URL and parameters
# base_url = "https://www.ncei.noaa.gov/access/services/data/v1"
# datasets_url = "?dataset=daily-summaries"
# station_url = "&stations=ASN00083035,ASN00083052,ASN00083068,ASN00083090,ASN00084010,ASN00084016,ASN00084017,ASN00084022,ASN00084030,ASN00084059,ASN00084073,ASN00084084,ASN00084085,ASN00084114,ASN00084121,ASN00084143,ASN00084144,ASN00084145,ASN00085062,ASN00085072,ASN00085092,ASN00085093,ASN00085106,ASN00085142,ASN00085193,ASN00085242,ASN00085244,ASN00085247,ASN00085279,ASN00085280,ASN00085291,ASN00085296,ASN00086133,ASN00089037"
# startdate_url = "&startDate=2016-01-01"
# enddate_url = "&endDate=2020-12-31"
# options_url = "&options=includeAttributes:true"
# data_format = "&format=json"

# # Construct the full URL
# full_url = f"{base_url}{datasets_url}{station_url}{startdate_url}{enddate_url}{options_url}{data_format}"
# token = "vflMliolsDnHblAoiNDDQgmocEipDpTQ"

# # Make the API call
# headers = {"Authorization": api_key}
# response = requests.get(full_url, headers=headers)

# # Check if the request was successful
# if response.status_code == 200:
#     # Convert the response content to JSON
#     api_json = response.json()
    
#     # Convert JSON to DataFrame
#     api_df = pd.DataFrame(api_json)
    
#     # Select desired columns
#     api_df = api_df[['DATE', 'STATION', 'TMIN', 'TMAX', 'PRCP']]
    
#     # Display the DataFrame
#     print(api_df.head())
# else:
#     print(f"Failed to retrieve data from the API. Status code: {response.status_code}")


# API Method2

In [ ]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get("API_KEY_MOP")

base_url = 'https://melbournetestbed.opendatasoft.com/api/explore/v2.1/catalog/datasets/'
dataset_id = 'footpath-steepness'
apikey = api_key
dataset_id = dataset_id
format = 'csv'

params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': apikey
}
url = f'{base_url}{dataset_id}/exports/{format}'
# GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    footpath_steepness = pd.read_csv(StringIO(url_content), delimiter=';')
    print(footpath_steepness.sample(10, random_state=999)) # Test
else:
    print(f'Request failed with status code {response.status_code}')

In [7]:
import requests
import pandas as pd
from io import StringIO
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get("API_KEY_MOP")

base_url = 'https://melbournetestbed.opendatasoft.com/api/explore/v2.1/catalog/datasets/'
dataset_id = 'pedestrian-counting-system-monthly-counts-per-hour'
apikey = api_key
dataset_id = dataset_id
format = 'csv'

params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': apikey
}
url = f'{base_url}{dataset_id}/exports/{format}'
# GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    pedestrian_count = pd.read_csv(StringIO(url_content), delimiter=';')
    print(pedestrian_count.sample(10, random_state=999)) # Test
    
else:
    print(f'Request failed with status code {response.status_code}')

       sensor_name                  timestamp  locationid  direction_1  \
299310    Col700_T  2023-06-11T00:00:00+00:00           9           72   
273077    SprFli_T  2024-01-16T19:00:00+00:00          75           30   
230538    Bou688_T  2023-08-22T21:00:00+00:00          58          794   
545967    FLDegC_T  2024-03-12T00:00:00+00:00          69          211   
41658     BouHbr_T  2023-07-06T06:00:00+00:00          10          275   
311686    BouBri_T  2023-06-03T18:00:00+00:00          57            2   
503749      ACMI_T  2024-02-27T12:00:00+00:00          72           10   
362060     Col12_T  2023-12-08T17:00:00+00:00          18          471   
90696         AG_T  2023-09-29T13:00:00+00:00          29           71   
340770      ACMI_T  2023-08-06T06:00:00+00:00          72          352   

        direction_2  total_of_directions                    location  
299310           93                  165  -37.81982992, 144.95102555  
273077           18                   48  -

In [8]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get("API_KEY_MOP")

base_url = 'https://melbournetestbed.opendatasoft.com/api/explore/v2.1/catalog/datasets/'
dataset_id = 'pedestrian-counting-system-sensor-locations'
apikey = api_key
dataset_id = dataset_id
format = 'csv'

params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': apikey
}
url = f'{base_url}{dataset_id}/exports/{format}'
# GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    pedestrian_sensor_locations = pd.read_csv(StringIO(url_content), delimiter=';')
    print(pedestrian_sensor_locations.sample(10, random_state=999)) # Test
else:
    print(f'Request failed with status code {response.status_code}')

     location_id                                 sensor_description  \
91           141   Awning of Nationwide Parking 474 Flinders Street   
37           114                  Shrine of rememberance-North-West   
0              1                         Bourke Street Mall (North)   
28            88                                         Micro-Labs   
97           153  narrm ngarrgu Library - Level 2 - Study Area L...   
26            84   Elizabeth St - Flinders St (East) - New footpath   
134          150      narrm ngarrgu Library - Level 1 Main Stairs B   
108           50                         Faraday St-Lygon St (West)   
99           159                              516 Elizeberth Street   
89           135  I-Hub Southern Cross Station - Bourke Street E...   

       sensor_name installation_date                         note  \
91         474Fl_T        2023-11-10                          NaN   
37       ShrinNW_T        2022-10-14                          NaN   
0         B

In [9]:
pedestrian_merged_data = pd.merge(pedestrian_count, pedestrian_sensor_locations, left_on='locationid', right_on='location_id', how='inner')
pedestrian_merged_data
# pedestrian_merged_data.info()
# print(type(pedestrian_merged_data.timestamp[0]))
# pedestrian_merged_data['timestamp'] = pd.to_datetime(pedestrian_merged_data['timestamp'])
# print(type(pedestrian_merged_data.timestamp[0]))

pedestrian_merged_data.sort_values(by='timestamp',ascending=False)
# print(pedestrian_merged_data)

,sensor_name_x,timestamp,locationid,direction_1_x,direction_2_x,total_of_directions,location_x,location_id,sensor_description,sensor_name_y,installation_date,note,location_type,status,direction_1_y,direction_2_y,latitude,longitude,location_y
390211,RMIT_T,2024-03-18T14:00:00+00:00,59,29,16,45,"-37.80825648, 144.96304859",59,Building 80 RMIT,RMIT_T,2019-02-13,NaN,Outdoor,A,North,South,-37.808256,144.963049,"-37.80825648, 144.96304859"
54948,Errol23_T,2024-03-18T14:00:00+00:00,87,1,0,1,"-37.80454949, 144.94921863",87,Errol St (West),Errol23_T,2022-05-20,NaN,Outdoor,A,North,South,-37.804549,144.949219,"-37.80454949, 144.94921863"
519374,King335_T,2024-03-18T14:00:00+00:00,132,17,10,27,"-37.81267639, 144.95386444",132,I-Hub Corner of King Street and Latrobe Street...,King335_T,2023-09-25,NaN,Outdoor,A,North,South,-37.812676,144.953864,"-37.81267639, 144.95386444"
455115,Errol20_T,2024-03-18T14:00:00+00:00,70,0,1,1,"-37.80456984, 144.94946228",70,Errol Street (East),Errol20_T,2020-10-12,NaN,Outdoor,A,North,South,-37.804570,144.949462,"-37.80456984, 144.94946228"
313658,Pel147_T,2024-03-18T14:00:00+00:00,46,6,0,6,"-37.80240719, 144.9615673",46,Pelham St (South),Pel147_T,2017-07-10,NaN,Outdoor,A,East,West,-37.802407,144.961567,"-37.80240719, 144.9615673"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139192,WebBN_T,2023-03-31T13:00:00+00:00,8,10,5,15,"-37.82293543, 144.9471751",8,Webb Bridge,WebBN_T,2009-03-24,NaN,Outdoor,A,North,South,-37.822935,144.947175,"-37.82293543, 144.9471751"
476967,HarEsB_T,2023-03-31T13:00:00+00:00,78,3,7,10,"-37.81471642, 144.9446508",78,Harbour Esplanade (West) - Bike Path,HarEsB_T,2021-03-30,NaN,Outdoor,A,North,South,-37.814716,144.944651,"-37.81471642, 144.9446508"
493014,Col12_T,2023-03-31T13:00:00+00:00,18,13,8,21,"-37.81344862, 144.97305353",18,Collins Place (North),Col12_T,2009-03-30,NaN,Outdoor,A,East,West,-37.813449,144.973054,"-37.81344862, 144.97305353"
247981,Lyg161_T,2023-03-31T13:00:00+00:00,31,34,81,115,"-37.80169681, 144.96658911",31,Lygon St (West),Lyg161_T,2013-10-10,"Pushbox Upgrade, 25/07/2023",Outdoor,A,North,South,-37.801697,144.966589,"-37.80169681, 144.96658911"


In [10]:
# Define the start and end dates for filtering
start_date = pd.to_datetime('2024-03-01').date()
end_date = pd.to_datetime('2024-03-10').date()

# Convert 'timestamp' column to datetime if it's not already
pedestrian_merged_data['timestamp'] = pd.to_datetime(pedestrian_merged_data['timestamp'])

# Extract date from 'timestamp' column
pedestrian_merged_data['date_only'] = pedestrian_merged_data['timestamp'].dt.date

# Filter the combined data DataFrame by date range
filtered_pedestrian_merged_data = pedestrian_merged_data[(pedestrian_merged_data['date_only'] >= start_date) & 
                                                         (pedestrian_merged_data['date_only'] <= end_date)]
filtered_pedestrian_merged_data.head(40)

,sensor_name_x,timestamp,locationid,direction_1_x,direction_2_x,total_of_directions,location_x,location_id,sensor_description,sensor_name_y,installation_date,note,location_type,status,direction_1_y,direction_2_y,latitude,longitude,location_y,date_only
5409,SprFli_T,2024-03-01 01:00:00+00:00,75,63,59,122,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-01
5410,SprFli_T,2024-03-01 07:00:00+00:00,75,68,49,117,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-01
5411,SprFli_T,2024-03-01 08:00:00+00:00,75,57,60,117,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-01
5524,SprFli_T,2024-03-01 19:00:00+00:00,75,8,3,11,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-01
5525,SprFli_T,2024-03-01 20:00:00+00:00,75,19,25,44,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-01
5526,SprFli_T,2024-03-02 02:00:00+00:00,75,74,62,136,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-02
5527,SprFli_T,2024-03-02 04:00:00+00:00,75,36,52,88,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-02
5528,SprFli_T,2024-03-02 05:00:00+00:00,75,45,89,134,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-02
5529,SprFli_T,2024-03-02 07:00:00+00:00,75,39,30,69,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-02
5530,SprFli_T,2024-03-02 22:00:00+00:00,75,37,31,68,"-37.81515276, 144.97467661",75,Spring St- Flinders st (West),SprFli_T,2020-12-18,NaN,Outdoor,A,North,South,-37.815153,144.974677,"-37.81515276, 144.97467661",2024-03-02


In [27]:
filtered_pedestrian_merged_data = filtered_pedestrian_merged_data.reindex(columns=['timestamp', 'latitude', 'longitude', 'direction_1_x', 'direction_2_x', 'total_of_directions', 'direction_1_y', 'direction_2_y', 'date_only'])
print(filtered_pedestrian_merged_data)
filtered_pedestrian_merged_data.head(40)


                       timestamp   latitude   longitude  direction_1_x  \
5409   2024-03-01 01:00:00+00:00 -37.815153  144.974677             63   
5410   2024-03-01 07:00:00+00:00 -37.815153  144.974677             68   
5411   2024-03-01 08:00:00+00:00 -37.815153  144.974677             57   
5524   2024-03-01 19:00:00+00:00 -37.815153  144.974677              8   
5525   2024-03-01 20:00:00+00:00 -37.815153  144.974677             19   
...                          ...        ...         ...            ...   
544433 2024-03-10 03:00:00+00:00 -37.825910  144.961860             55   
544434 2024-03-10 06:00:00+00:00 -37.825910  144.961860             87   
544435 2024-03-10 07:00:00+00:00 -37.825910  144.961860             99   
544436 2024-03-10 20:00:00+00:00 -37.825910  144.961860             32   
544437 2024-03-10 22:00:00+00:00 -37.825910  144.961860             94   

        direction_2_x  total_of_directions  cdirection_1_y direction_2_y  \
5409               59              

,timestamp,latitude,longitude,direction_1_x,direction_2_x,total_of_directions,cdirection_1_y,direction_2_y,date_only
5409,2024-03-01 01:00:00+00:00,-37.815153,144.974677,63,59,122,NaN,South,2024-03-01
5410,2024-03-01 07:00:00+00:00,-37.815153,144.974677,68,49,117,NaN,South,2024-03-01
5411,2024-03-01 08:00:00+00:00,-37.815153,144.974677,57,60,117,NaN,South,2024-03-01
5524,2024-03-01 19:00:00+00:00,-37.815153,144.974677,8,3,11,NaN,South,2024-03-01
5525,2024-03-01 20:00:00+00:00,-37.815153,144.974677,19,25,44,NaN,South,2024-03-01
5526,2024-03-02 02:00:00+00:00,-37.815153,144.974677,74,62,136,NaN,South,2024-03-02
5527,2024-03-02 04:00:00+00:00,-37.815153,144.974677,36,52,88,NaN,South,2024-03-02
5528,2024-03-02 05:00:00+00:00,-37.815153,144.974677,45,89,134,NaN,South,2024-03-02
5529,2024-03-02 07:00:00+00:00,-37.815153,144.974677,39,30,69,NaN,South,2024-03-02
5530,2024-03-02 22:00:00+00:00,-37.815153,144.974677,37,31,68,NaN,South,2024-03-02


In [12]:
pedestrian_latitude = filtered_pedestrian_merged_data['latitude'].values[0]
pedestrian_longitude = filtered_pedestrian_merged_data['longitude'].values[0]

In [17]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

def get_climate_data(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": pedestrian_latitude,
        "longitude": pedestrian_longitude,
        "current": "relative_humidity_2m",
        "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "showers", "weather_code", "uv_index"],
        "past_days": 92
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process hourly data
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
    hourly_rain = hourly.Variables(3).ValuesAsNumpy()
    hourly_showers = hourly.Variables(4).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(5).ValuesAsNumpy()
    hourly_uv_index = hourly.Variables(6).ValuesAsNumpy()

    hourly_data = {
        "latitude": latitude,
        "longitude": longitude,
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature_2m": hourly_temperature_2m,
        "relative_humidity_2m": hourly_relative_humidity_2m,
        "precipitation": hourly_precipitation,
        "rain": hourly_rain,
        "showers": hourly_showers,
        "weather_code": hourly_weather_code,
        "uv_index": hourly_uv_index
    }

    hourly_dataframe = pd.DataFrame(data=hourly_data)
    return hourly_dataframe


# Initialize an empty list to store all climate dataframes
all_climate_data = []

# Iterate over each location and retrieve climate data
for index, row in filtered_pedestrian_merged_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
   
    climate_data = get_climate_data(latitude, longitude)
    all_climate_data.append(climate_data)

# Concatenate all climate dataframes into a single dataframe
climate_data_combined = pd.concat(all_climate_data, ignore_index=True)

# Print the combined climate data

climate_data_combined = climate_data_combined.rename(columns={'date': 'timestamp'})
climate_data_combined

,latitude,longitude,timestamp,temperature_2m,relative_humidity_2m,precipitation,rain,showers,weather_code,uv_index
0,-37.815153,144.974677,2023-12-28 00:00:00+00:00,19.065001,81.0,0.0,0.0,0.0,3.0,3.85
1,-37.815153,144.974677,2023-12-28 01:00:00+00:00,18.965000,81.0,0.0,0.0,0.0,3.0,2.70
2,-37.815153,144.974677,2023-12-28 02:00:00+00:00,18.715000,80.0,0.0,0.0,0.0,3.0,2.05
3,-37.815153,144.974677,2023-12-28 03:00:00+00:00,19.015001,75.0,0.0,0.0,0.0,3.0,1.25
4,-37.815153,144.974677,2023-12-28 04:00:00+00:00,18.865000,73.0,0.0,0.0,0.0,3.0,1.15
...,...,...,...,...,...,...,...,...,...,...
45806899,-37.825910,144.961860,2024-04-04 19:00:00+00:00,11.264999,84.0,0.0,0.0,0.0,1.0,0.00
45806900,-37.825910,144.961860,2024-04-04 20:00:00+00:00,11.315000,86.0,0.0,0.0,0.0,1.0,0.00
45806901,-37.825910,144.961860,2024-04-04 21:00:00+00:00,11.815000,84.0,0.0,0.0,0.0,1.0,0.00
45806902,-37.825910,144.961860,2024-04-04 22:00:00+00:00,13.065000,78.0,0.0,0.0,0.0,2.0,0.45


In [28]:
merged_df = pd.merge(filtered_pedestrian_merged_data, climate_data_combined, on=['timestamp', 'latitude', 'longitude'], how='left')
merged_df = merged_df.drop_duplicates()
print(merged_df)
merged_df.head(40)

                        timestamp   latitude   longitude  direction_1_x  \
0       2024-03-01 01:00:00+00:00 -37.815153  144.974677             63   
156     2024-03-01 07:00:00+00:00 -37.815153  144.974677             68   
312     2024-03-01 08:00:00+00:00 -37.815153  144.974677             57   
468     2024-03-01 19:00:00+00:00 -37.815153  144.974677              8   
624     2024-03-01 20:00:00+00:00 -37.815153  144.974677             19   
...                           ...        ...         ...            ...   
4556093 2024-03-10 03:00:00+00:00 -37.825910  144.961860             55   
4556239 2024-03-10 06:00:00+00:00 -37.825910  144.961860             87   
4556385 2024-03-10 07:00:00+00:00 -37.825910  144.961860             99   
4556531 2024-03-10 20:00:00+00:00 -37.825910  144.961860             32   
4556677 2024-03-10 22:00:00+00:00 -37.825910  144.961860             94   

         direction_2_x  total_of_directions  cdirection_1_y direction_2_y  \
0                   59

,timestamp,latitude,longitude,direction_1_x,direction_2_x,total_of_directions,cdirection_1_y,direction_2_y,date_only,temperature_2m,relative_humidity_2m,precipitation,rain,showers,weather_code,uv_index
0,2024-03-01 01:00:00+00:00,-37.815153,144.974677,63,59,122,NaN,South,2024-03-01,24.165001,49.0,0.0,0.0,0.0,3.0,6.10
156,2024-03-01 07:00:00+00:00,-37.815153,144.974677,68,49,117,NaN,South,2024-03-01,21.965000,63.0,0.0,0.0,0.0,0.0,3.10
312,2024-03-01 08:00:00+00:00,-37.815153,144.974677,57,60,117,NaN,South,2024-03-01,20.265001,66.0,0.0,0.0,0.0,0.0,1.40
468,2024-03-01 19:00:00+00:00,-37.815153,144.974677,8,3,11,NaN,South,2024-03-01,14.714999,85.0,0.0,0.0,0.0,2.0,0.00
624,2024-03-01 20:00:00+00:00,-37.815153,144.974677,19,25,44,NaN,South,2024-03-01,14.514999,87.0,0.0,0.0,0.0,2.0,0.00
780,2024-03-02 02:00:00+00:00,-37.815153,144.974677,74,62,136,NaN,South,2024-03-02,19.815001,51.0,0.0,0.0,0.0,2.0,5.15
936,2024-03-02 04:00:00+00:00,-37.815153,144.974677,36,52,88,NaN,South,2024-03-02,20.365000,49.0,0.0,0.0,0.0,1.0,7.40
1092,2024-03-02 05:00:00+00:00,-37.815153,144.974677,45,89,134,NaN,South,2024-03-02,20.265001,49.0,0.0,0.0,0.0,1.0,6.40
1248,2024-03-02 07:00:00+00:00,-37.815153,144.974677,39,30,69,NaN,South,2024-03-02,19.315001,52.0,0.0,0.0,0.0,1.0,3.05
1404,2024-03-02 22:00:00+00:00,-37.815153,144.974677,37,31,68,NaN,South,2024-03-02,12.565000,64.0,0.0,0.0,0.0,2.0,1.15


In [ ]:
# # Extract date and time separately
# climate_data_combined['date'] = pd.to_datetime(climate_data_combined['date'])
# climate_data_combined['date_only'] = climate_data_combined['date'].dt.date
# climate_data_combined['time_only'] = climate_data_combined['date'].dt.time

# # Print the combined climate data with separate date and time columns
# print(climate_data_combined[['latitude','longitude','date_only', 'time_only', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'rain', 'showers', 'weather_code', 'uv_index']].head(30))
# climate_data_combined
# climate_data_combined.info()


In [ ]:
# # Convert 'date_only' to datetime
# climate_data_combined['date_only'] = pd.to_datetime(climate_data_combined['date_only'])


In [ ]:
# selected_columns = climate_data_combined[['date_only','latitude', 'longitude', 'time_only', 'temperature_2m']]
# # Create a new DataFrame with selected columns
# temperature_df = pd.DataFrame(selected_columns)

# # Pivot the DataFrame to a wider format with hours as columns
# wider_temperature_df = temperature_df.pivot(index=['latitude', 'longitude', 'date_only'], columns='time_only', values='temperature_2m')

# # Reset the index to make 'latitude', 'longitude', and 'date' as columns
# wider_temperature_df.reset_index(inplace=True)

# # Display the wider format DataFrame
# print(wider_temperature_df)
# data_types = wider_temperature_df.dtypes

# # Print the data types
# print(data_types)

In [ ]:
# # Define the start and end dates for filtering
# start_date = pd.to_datetime('2023-12-30').date()
# end_date = pd.to_datetime('2024-03-25').date()

# # Convert 'date_only' column to datetime.date objects if it's not already
# wider_temperature_df['date_only'] = pd.to_datetime(wider_temperature_df['date_only']).dt.date

# # Filter the combined climate data DataFrame by date range
# filtered_climate_data = wider_temperature_df[(wider_temperature_df['date_only'] >= start_date) & 
#                                               (wider_temperature_df['date_only'] <= end_date)]

# # Print the filtered climate data
# print(filtered_climate_data[['latitude', 'longitude', 'date_only', 'time_only', 'temperature_2m', 
#                              'relative_humidity_2m', 'precipitation', 'rain', 'showers', 
#                              'weather_code', 'uv_index']].head(30))




#Dinuk Edited
_**DELETE BEFORE PUBLISHING**_

_This is a template also containing the style guide for use cases. The styling uses the use-case css when uploaded to the website, which will not be visible on your local machine._

_Change any text marked with {} and delete any cells marked DELETE_

***

In [ ]:
# DELETE BEFORE PUBLISHING
# This is just here so you can preview the styling on your local machine

from IPython.core.display import HTML
HTML("""
<style>
.usecase-title, .usecase-duration, .usecase-section-header {
    padding-left: 15px;
    padding-bottom: 10px;
    padding-top: 10px;
    padding-right: 15px;
    background-color: #0f9295;
    color: #fff;
}

.usecase-title {
    font-size: 1.7em;
    font-weight: bold;
}

.usecase-authors, .usecase-level, .usecase-skill {
    padding-left: 15px;
    padding-bottom: 7px;
    padding-top: 7px;
    background-color: #baeaeb;
    font-size: 1.4em;
    color: #121212;
}

.usecase-level-skill  {
    display: flex;
}

.usecase-level, .usecase-skill {
    width: 50%;
}

.usecase-duration, .usecase-skill {
    text-align: right;
    padding-right: 15px;
    padding-bottom: 8px;
    font-size: 1.4em;
}

.usecase-section-header {
    font-weight: bold;
    font-size: 1.5em;
}

.usecase-subsection-header, .usecase-subsection-blurb {
    font-weight: bold;
    font-size: 1.2em;
    color: #121212;
}

.usecase-subsection-blurb {
    font-size: 1em;
    font-style: italic;
}
</style>
""")

<div class="usecase-title">{Use Case Name}</div>

<div class="usecase-authors"><b>Authored by: </b> {Author/s}</div>

<div class="usecase-duration"><b>Duration:</b> {90} mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>{Intermediate}</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>{Python, and add any more skills needed}</div>
</div>

<div class="usecase-section-header">Scenario</div>

{Using User Story format, write a description of the problem you are trying to solve for this use case.}

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- {list the skills demonstrated in your use case}

<div class="usecase-section-header">{Heading for introduction or background relating to problem}</div>

{Write your introduction here. Keep it concise. We're not after "War and Peace" but enough background information to inform the reader on the rationale for solving this problem or background non-technical information that helps explain the approach. You may also wish to give information on the datasets, particularly how to source those not being imported from the client's open data portal.}



***

_**DELETE BEFORE PUBLISHING**_

## Style guide for use cases

### Headers

For styling within your markdown cells, there are two choices you can use for headers.

1) You can use HTML classes specific to the use case styling:

```<p class="usecase-subsection-header">This is a subsection header.</p>```

<p style="font-weight: bold; font-size: 1.2em;">This is a subsection header.</p>

```<p class="usecase-subsection-blurb">This is a blurb header.</p>```

<p style="font-weight: bold; font-size: 1em; font-style:italic;">This is a blurb header.</p>


2) Or if you like you can use the markdown header styles:

```# for h1```

```## for h2```

```### for h3```

```#### for h4```

```##### for h5```

## Plot colour schemes

General advice:
1. Use the same colour or colour palette throughout your notebook, unless variety is necessary
2. Select a palette based on the type of data being represented
3. Consider accessibility (colourblindness, low vision)

#### 1) If all of your plots only use 1-2 colors use one of the company style colors:

| Light theme | Dark Theme |
|-----|-----|
|<p style="color:#2af598;">#2af598</p>|<p style="color:#08af64;">#08af64</p>|
|<p style="color:#22e4ac;">#22e4ac</p>|<p style="color:#14a38e;">#14a38e</p>|
|<p style="color:#1bd7bb;">#1bd7bb</p>|<p style="color:#0f9295;">#0f9295</p>|
|<p style="color:#14c9cb;">#14c9cb</p>|<p style="color:#056b8a;">#056b8a</p>|
|<p style="color:#0fbed8;">#0fbed8</p>|<p style="color:#121212;">#121212</p>|
|<p style="color:#08b3e5;">#08b3e5</p>||


#### 2) If your plot needs multiple colors, choose an appropriate palette using either of the following tutorials:
- https://seaborn.pydata.org/tutorial/color_palettes.html
- https://matplotlib.org/stable/tutorials/colors/colormaps.html

#### 3) Consider accessibility as well.

For qualitative plotting Seaborn's 'colorblind' palette is recommended. For maps with sequential or diverging it is recommended to use one of the Color Brewer schemes which can be previewed at https://colorbrewer2.org/.

If you want to design your own colour scheme, it should use the same principles as Cynthia Brewer's research (with variation not only in hue but also, saturation or luminance).

### References

Be sure to acknowledge your sources and any attributions using links or a reference list.

If you have quite a few references, you might wish to have a dedicated section for references at the end of your document, linked using footnote style numbers.

You can connect your in-text reference by adding the number with a HTML link: ```<a href="#fn-1">[1]</a>```

and add a matching ID in the reference list using the ```<fn>``` tag: ```<fn id="fn-1">[1] Author (Year) _Title_, Publisher, Publication location.</fn>```